## Step 1: Import Packages and Declare Constants

In [21]:
import boto3
import sagemaker
import datetime as dt
import pandas as pd

In [22]:
#Replace this value with the S3 Bucket Created
default_bucket = "storeretaildatasetchurnpredictiontest"

In [23]:
region = boto3.Session().region_name
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()
sklearn_processor_version="0.23-1"
model_package_group_name="ChurnModelPackageGroup"
pipeline_name= "ChurnModelSMPipeline"
clarify_image = sagemaker.image_uris.retrieve(framework='sklearn',version=sklearn_processor_version,region=region)

## Step 2: Generate Baseline Dataset

Baseline Data will be used as part of SageMaker Clarify Step to generate SHAP Values 

In [24]:
def preprocess_data(file_path):
    df = pd.read_csv(file_path)
    ## Convert to datetime columns
    df["firstorder"]=pd.to_datetime(df["firstorder"],errors='coerce')
    df["lastorder"] = pd.to_datetime(df["lastorder"],errors='coerce')
    ## Drop Rows with null values
    df = df.dropna()
    ## Create Column which gives the days between the last order and the first order
    df["first_last_days_diff"] = (df['lastorder']-df['firstorder']).dt.days
    ## Create Column which gives the days between when the customer record was created and the first order
    df['created'] = pd.to_datetime(df['created'])
    df['created_first_days_diff']=(df['created']-df['firstorder']).dt.days
    ## Drop Columns
    df.drop(['custid','created','firstorder','lastorder'],axis=1,inplace=True)
    ## Apply one hot encoding on favday and city columns
    df = pd.get_dummies(df,prefix=['favday','city'],columns=['favday','city'])
    return df

In [38]:
baseline_data = preprocess_data(f"s3://{default_bucket}/data/storedata_total.csv")
baseline_data.pop("retained")
baseline_sample = baseline_data.sample(frac=0.0002)

In [39]:
pd.DataFrame(baseline_sample).to_csv(f"s3://{default_bucket}/data/baseline.csv",header=False,index=False)


## Step 3: Generate Batch Dataset

In [40]:
batch_data = preprocess_data(f"s3://{default_bucket}/data/storedata_total.csv")
batch_data.pop("retained")
batch_sample = batch_data.sample(frac=0.2)

In [42]:
pd.DataFrame(batch_sample).to_csv(f"s3://{default_bucket}/data/batch.csv",header=False,index=False)

## Step 4: Copy Data and Scripts to S3 Bucket

In [45]:
#already did in previous steps
# s3_client = boto3.resource('s3')
# s3_client.Bucket(default_bucket).upload_file("data/storedata_total.csv","data/storedata_total.csv")
# s3_client.Bucket(default_bucket).upload_file("data/batch.csv","data/batch/batch.csv")
# s3_client.Bucket(default_bucket).upload_file("data/baseline.csv","input/baseline/baseline.csv")

In [46]:
s3_client.Bucket(default_bucket).upload_file("pipelines/customerchurn/preprocess.py","input/code/preprocess.py")
s3_client.Bucket(default_bucket).upload_file("pipelines/customerchurn/evaluate.py","input/code/evaluate.py")
s3_client.Bucket(default_bucket).upload_file("pipelines/customerchurn/generate_config.py","input/code/generate_config.py")

## Step 5: Get the Pipeline Instance

In [47]:
from pipelines.customerchurn.pipeline import get_pipeline

pipeline = get_pipeline(
    region = region,
    role=role,
    default_bucket=default_bucket,
    model_package_group_name=model_package_group_name,
    pipeline_name=pipeline_name,
    custom_image_uri=clarify_image,
    sklearn_processor_version=sklearn_processor_version
)

INFO:sagemaker.image_uris:Defaulting to only available Python version: py3
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: 1.0.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [48]:
pipeline.definition()

'{"Version": "2020-12-01", "Metadata": {}, "Parameters": [{"Name": "ProcessingInstanceType", "Type": "String", "DefaultValue": "ml.m5.xlarge"}, {"Name": "ProcessingInstanceCount", "Type": "Integer", "DefaultValue": 1}, {"Name": "TrainingInstanceType", "Type": "String", "DefaultValue": "ml.m5.xlarge"}, {"Name": "InputData", "Type": "String", "DefaultValue": "s3://storeretaildatasetchurnpredictiontest/data/storedata_total.csv"}, {"Name": "BatchData", "Type": "String", "DefaultValue": "s3://storeretaildatasetchurnpredictiontest/data/batch/batch.csv"}], "PipelineExperimentConfig": {"ExperimentName": {"Get": "Execution.PipelineName"}, "TrialName": {"Get": "Execution.PipelineExecutionId"}}, "Steps": [{"Name": "ChurnModelProcess", "Type": "Processing", "Arguments": {"ProcessingResources": {"ClusterConfig": {"InstanceType": {"Get": "Parameters.ProcessingInstanceType"}, "InstanceCount": {"Get": "Parameters.ProcessingInstanceCount"}, "VolumeSizeInGB": 30}}, "AppSpecification": {"ImageUri": "7833

## Step 5: Submit the pipeline to SageMaker and start execution

In [49]:
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:ap-southeast-2:448049806778:pipeline/ChurnModelSMPipeline',
 'ResponseMetadata': {'RequestId': '568159a9-f7cc-403e-b552-17e52739d4df',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '568159a9-f7cc-403e-b552-17e52739d4df',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '93',
   'date': 'Sat, 08 Feb 2025 20:58:59 GMT'},
  'RetryAttempts': 0}}

Start Pipeline Execution

In [50]:
execution = pipeline.start()

Now we describe execution instance and list the steps in the execution to find out more about the execution.

In [51]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:ap-southeast-2:448049806778:pipeline/ChurnModelSMPipeline',
 'PipelineExecutionArn': 'arn:aws:sagemaker:ap-southeast-2:448049806778:pipeline/ChurnModelSMPipeline/execution/4sow9e12q57y',
 'PipelineExecutionDisplayName': 'execution-1739048344179',
 'PipelineExecutionStatus': 'Executing',
 'PipelineExperimentConfig': {'ExperimentName': 'churnmodelsmpipeline',
  'TrialName': '4sow9e12q57y'},
 'CreationTime': datetime.datetime(2025, 2, 8, 20, 59, 4, 97000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2025, 2, 8, 20, 59, 4, 97000, tzinfo=tzlocal()),
 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:ap-southeast-2:448049806778:user-profile/d-uvlaog8rhyr6/Nathan',
  'UserProfileName': 'Nathan',
  'DomainId': 'd-uvlaog8rhyr6',
  'IamIdentity': {'Arn': 'arn:aws:sts::448049806778:assumed-role/AmazonSageMaker-ExecutionRole-20241028T222872/SageMaker',
   'PrincipalId': 'AROAWQUOZPW5NNU23VJI2:SageMaker'}},
 'LastModifiedBy': {'UserProfileArn': 'arn:

We can list the execution steps to check out the status and artifacts:

In [52]:
execution.list_steps()

[{'StepName': 'ChurnModelProcess',
  'StartTime': datetime.datetime(2025, 2, 8, 20, 59, 5, 290000, tzinfo=tzlocal()),
  'StepStatus': 'Executing',
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:ap-southeast-2:448049806778:processing-job/pipelines-4sow9e12q57y-ChurnModelProcess-42HwCKywHj'}},
  'AttemptCount': 1}]